In [251]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd 
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
iris = datasets.load_iris()

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [4]:
data1 = pd.DataFrame(iris.data)

In [5]:
data1.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w']

In [6]:
label1 = pd.DataFrame(iris.target)

In [47]:
label1.columns = ['label']

In [66]:
label1.label.value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [48]:
train_x, test_x, train_y, test_y = train_test_split(data1.values, label1.values, test_size=0.3, random_state=32)

In [266]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((105, 4), (45, 4), (105, 1), (45, 1))

In [287]:
%%time
test_data = xgb.DMatrix(test_x, label=test_y)
train_data = xgb.DMatrix(train_x, label=train_y)
xgb_params = {
    'eta':0.1,
    'silent':True,
    'objective':'multi:softprob',
    'num_class':3,
    'max_depth':3
}
runs = 20
model = xgb.train(xgb_params, train_data, runs)
test_pre = model.predict(test_data)
test_pre_1 = np.argmax(test_pre,axis=1)
test_pre_1.shape, test_pre.shape
precision_score(test_y, test_pre_1, average='macro'),  recall_score(test_y, test_pre_1, average='macro')

Wall time: 20 ms


In [249]:
model2 = XGBClassifier(learning_rate=0.1, objective='multi:softprob', max_depth=3, seed=27, n_estimators=100)
model2.fit(train_x,train_y.ravel())
test2_pre = model2.predict(test_x)
f1_score(test_y, test2_pre, average='macro')

C:\ProgramData\Anaconda3\envs\nlp\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1.0

In [286]:
%%time
train_data = lgb.Dataset(train_x, train_y.ravel())
test_data = lgb.Dataset(test_x, test_y.ravel())
params = {
    'boosting_type':'gbdt',
    'objective':'multiclass',
    'metric':'multi_error',
    'verbos':1,
    'num_class':3
}
model = lgb.train(params, train_data)
test_pre = model.predict(test_x)
test_pre = np.argmax(test_pre, axis=1)
f1_score(test_y, test_pre, average='macro')

Wall time: 313 ms


In [290]:
params = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':31,
    'max_depth':11,
    'objective':'multiclass',
    'n_estimators':300
}
model = LGBMClassifier(**params)
model.fit(train_x, train_y)
test_pre = model.predict(test_x)
f1_score(test_y, test_pre, average='macro')

C:\ProgramData\Anaconda3\envs\nlp\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\nlp\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\nlp\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1.0